In [ ]:
### FOR COLAB ONLY ###

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

In [ ]:
#cd /content/drive/My Drive

In [ ]:
#!pip install musdb

In [ ]:
### IMPORTS ###

In [ ]:
import musdb
import os
import torch
import torch.optim as optim
import torch.nn as nn
from moviepy.editor import AudioFileClip, ColorClip
from scipy.io.wavfile import write
import pickle
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
from modules.model import ssnet
from modules.build_dataset import process_audio

In [ ]:
### BUILD DATASET ###

# mus.audio : The mixture
# mus.stems[0] : The mixture
# mus.stems[1] : The drums
# mus.stems[2] : The bass
# mus.stems[3] : Rest of the accompaniment
# mus.stems[4] : The vocals

# TODO: We want the input to be the mixture (stems[1] or mus.audio)
# --> target has to be everything but vocals, thus: Find a way to mix stems[1], stems[2], stems[3]
# --> temporarly here the target is stems[4] (vocals)


In [ ]:
# Set the MUSDB_PATH environment variable
os.environ['MUSDB_PATH'] = 'data/musdb18'
# Initialize the musdb datasets
mus_train = musdb.DB(subsets="train", split='train')
mus_valid = musdb.DB(subsets="train", split='valid')
mus_test = musdb.DB(subsets="test")

In [ ]:
FRAME_LENGTH = 30000
MAX_AUDIO_LENGTH = 9000000

In [ ]:
X_train, y_train = process_audio(mus_train, tag='train', frame_length=FRAME_LENGTH, max_audio_length=MAX_AUDIO_LENGTH)
#X_train, y_train = pickle.load(open('train_set.pkl', 'rb'))

X_valid, y_valid = process_audio(mus_valid, tag='valid', frame_length=FRAME_LENGTH, max_audio_length=MAX_AUDIO_LENGTH)
#X_valid, y_valid = pickle.load(open('valid_set.pkl', 'rb'))

X_test, y_test = process_audio(mus_test, tag='test', frame_length=FRAME_LENGTH, max_audio_length=MAX_AUDIO_LENGTH)
#X_test, y_test = pickle.load(open('test_set.pkl', 'rb'))

In [ ]:
!python train_script.py train_split valid_split 8 10 0.001 0.0001 'run'